# 🗣️ Talk to your Database 🗄️

Before we look at talking to unstructured documents, let's look at a quick small example of how one might "talk to" structured data.

The LangChain website has an example which demonstrates this:
- https://python.langchain.com/docs/use_cases/tabular/sqlite

We'll model this notebook after that example, using the classic "Northwind" database used by Microsoft to show database examples.

> The SQLite3 port of the "Northwind" database which we'll be using is provided by this MIT licensed repository:
> - https://github.com/jpwhite3/northwind-SQLite3

Conveniently (_or inconveniently_) I cannot find the source code for the LangChain SQL example, so we will create it ourselves!

## Providing the LLM with your Schema

The cell below runs a script that, given a connection string to a database, will print and store the schema of that database as JSON.

> Note: this notebook uses the `peewee` Python SQL ORM due to its simplicity _(very small amount of code required!)_
> - https://docs.peewee-orm.com/en/latest/
> 
> `peewee` supports: SQLite, MySQL, and PostgreSQL _(but the SQL in this example is specifically for SQLite)_
> 
> _Note: We love SQLAlchemy as well, but it requires a bit more boilerplate code than `peewee`_



In [ ]:
# Set this to any SQLite database connection string
db_connection_string = "sqlite:///northwind.sqlite3"

# If you plan on using MySQL or PostgreSQL, update these SQL statements below
# to ones which are compatible with your database.
SQL_get_all_table_names = "SELECT name FROM sqlite_master WHERE type='table';"
SQL_get_table_info = "PRAGMA table_info('{table_name}');"

In [ ]:
import peewee
from playhouse.db_url import connect as connect_to_database
import json

database: peewee.Database = connect_to_database(db_connection_string)

database_schema = {}

for table in database.execute_sql(SQL_get_all_table_names).fetchall():
    table_name = table[0]
    schema = database.execute_sql(SQL_get_table_info.format(table_name=table_name)).fetchall()
    database_schema[table_name] = {column[1]: column[2] for column in schema}

database_schema_json = json.dumps(database_schema)

In [ ]:
# Print out the database schema
print(json.dumps(database_schema, indent=4))

In [13]:
# This cell prints out an approximate token count of the JSON
# based on tokenizer of GPT 3.5
import tiktoken
print(f"The JSON schema is approximately {len(tiktoken.encoding_for_model('gpt-3.5-turbo').encode(database_schema_json))} tokens")

The JSON schema is approximately 666 tokens


## Asking the LLM for a SQL query

Now that we have the...